In [50]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
import copy
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [51]:
import os,glob
path0 = 'tsvfiles/'
df = spark.read.csv(
    os.path.join(path0,"SUNM.tsv"), sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0558,well i vote out and then when you you know when you've do it and er
S0517,do vote out
S0558,yeah
S0517,yeah
S0558,and i think have this feel you know on occasion i've have these feel and i think oh i dunno if i've do the right thing


In [52]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(len(stops))
filled_p = ['ah','oh','eh','er','erm','mm','mhm','huh','uhu']
for p in filled_p:
    stops.add(p)
print(stops)
print(len(stops))

179
{'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', "hasn't", 'huh', 'then', 'both', 'so', 're', 'don', 'm', 'as', 'any', 'mightn', 'after', 'you', 'wouldn', 'why', 'been', 'where', 'by', "isn't", 'yourself', 'wasn', 'a', 'er', "haven't", 'did', "hadn't", 'uhu', 'their', 'hasn', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', "you'll", 'yourselves', 'that', 'my', 'what', 'to', 'd', 'not', "won't", "couldn't", 'own', 'there', 'this', 'mhm', 'each', 'all', 'haven', 'more', 'me', 've', 'weren', 'which', 'himself', 'nor', 'other', "shouldn't", 'who', "should've", 'erm', 'same', 'at', 'such', 't', 'up', 'than', 'mm', 'can', "you've", 'too', 'ah', 'these', 'while', "wasn't", 'ourselves', 'before', 'i', 'he', "didn't", 'our', 'its', 'but', 'with', "wouldn't", 'those', 'because', 'the', 'y', 'shouldn', 'it', 'mustn', 'hers', 'just', 'doesn', 'ain', 'between', 'over', 'had', 'aren', "mightn't", 'does', 'have', 'and', 'or', 'some', "mustn't", 'only', 'won', 'when', 'needn', '

In [53]:
import xmltodict
def extract_time(f):
    with open(f) as fd:
        doc = xmltodict.parse(fd.read())
    s_time = doc['text']['header']['rec_length']
    s_seconds = (int(s_time[0])*3600)+ (int(s_time[2])*10+int(s_time[3]))*60 + int(s_time[5])*10+int(s_time[6])
    return s_seconds

s = extract_time('S376.xml')
print(s)

2026


In [54]:
def extract_speakers_n(df1, n):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0] not in speaker:
            speaker.append(row[0])
        if len(speaker) == n:
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [55]:
def extract_dialogues_dic_n(rdd1, speaker):
    n = len(speaker)
    dataC = rdd1.collect()
    speaker_dic = [{} for x in range(0,n)]
    
    
    for i in range(0,len(dataC)):
        if dataC[i][0] in speaker:
            p = speaker.index(dataC[i][0])
            if dataC[i][1]!=None:
                speaker_dic[p][i] = dataC[i][1].split()
    
    return speaker, speaker_dic

In [56]:
def get_vo(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            
            for token in tokens:
                token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    
    countand = 0
    if n==2:
        for token0 in speaker_vo[0]:
            if token0 in speaker_vo[1]:
                countand = countand+1
    if n==3:
        for token0 in speaker_vo[0]:
            if (token0 in speaker_vo[1]) and (token0 in speaker_vo[2]):
                countand = countand+1
    if n==4:
        for token0 in speaker_vo[0]:
            if (token0 in speaker_vo[1]) and (token0 in speaker_vo[2]) and (token0 in speaker_vo[3]):
                countand = countand+1
    countor = len(voor)
    overlap = countand/countor
    vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap} 
    return vo_group
            

In [57]:
def get_vo_pairwise(speaker, speaker_dic):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            for token in tokens:
                token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        for s_i in range(0,n-1):
            for s_j in range(s_i+1,n):
                for token in speaker_vo[s_i]:
                    if token in speaker_vo[s_j]:
                        countand_list[x].append(token)
                x = x+1
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_s2": len(countand_list[0]), "s1_and_s3": len(countand_list[1]),"s2_and_s3": len(countand_list[2])}
            
            
    if n==4:
        countand_list = [[] for i in range(0,6)]
        x = 0
        for s_i in range(0,n-1):
            for s_j in range(s_i+1,n):
                for token in speaker_vo[s_i]:
                    if token in speaker_vo[s_j]:
                        countand_list[x].append(token)
                x = x+1
        countand = 0               
        for c in range(0,6):
            countand = countand + len(countand_list[c])
        countor = len(voor)
        overlap = countand/(6*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_s2": len(countand_list[0]), "s1_and_s3": len(countand_list[1]),"s1_and_s4": len(countand_list[2]),\
                   "s2_and_s3": len(countand_list[3]),"s2_and_s4": len(countand_list[4]),"s3_and_s4": len(countand_list[5])}
    
    
    return vo_group

In [58]:
def get_vo_average_speaker(speaker, speaker_dic, ex_stop):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            token_n = token_n + len(tokens)
            for token in tokens:
#                 token_n = token_n+1
                if token not in voor:
                    voor.append(token)
                if token not in speaker_vo[i]:
                    speaker_vo[i].append(token)
    if n==2:
        voand_2 = []
        
        if ex_stop == True:
            for s0 in speaker_vo[0]:
                if s0 in speaker_vo[1]:
                    if (s0 not in voand_2) and (s0 not in stops):
                        voand_2.append(s0)
                    
        countand = len(voand_2)
        countor = len(voor)
        overlap = countand/countor
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap}
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        
        if ex_stop == True:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if (token not in countand_list[x]) and (token not in stops):
                                    countand_list[x].append(token)
                x = x+1
                
        else:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if token not in countand_list[x]:
                                    countand_list[x].append(token)
                x = x+1
            
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2])}
            
            
    if n==4:
        countand_list = [[] for i in range(0,4)]
        x = 0
        
        if ex_stop == True:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if (token not in countand_list[x]) and (token not in stops):
                                    countand_list[x].append(token)
                x = x+1
                
        else:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if token not in countand_list[x]:
                                    countand_list[x].append(token)
                x = x+1
            
        countand = 0               
        for c in range(0,4):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(4*countor)

        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "s4_and_other": len(countand_list[3])}
    
    
    return vo_group

In [72]:
def get_vo_average_speaker_in_window(speaker, speaker_dic, start, end, ex_stop):
    n = len(speaker)
    voand = []
    voor = []
    speaker_vo = [[] for x in range(0,n)]
    vo_group = {}
    token_n = 0
    
    for i in range(0,n):
        for k,tokens in speaker_dic[i].items():
            if k>=start and k<end:
                token_n = token_n + len(tokens)
                for token in tokens:
#                 token_n = token_n+1
                    if token not in voor:
                        voor.append(token)
                    if token not in speaker_vo[i]:
                        speaker_vo[i].append(token)
    
    if n==2:
        voand_2 = []
        
        if ex_stop == True:
            for s0 in speaker_vo[0]:
                if s0 in speaker_vo[1]:
                    if (s0 not in voand_2) and (s0 not in stops):
                        voand_2.append(s0)
                    
        countand = len(voand_2)
        countor = len(voor)
        overlap = countand/countor
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap, "begin":start, "end":end}
    
    
    
    if n==3:
        countand_list = [[] for j in range(0,3)]
        x = 0
        if ex_stop == True:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if (token not in countand_list[x]) and (token not in stops):
                                    countand_list[x].append(token)
                x = x+1
                
        else:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if token not in countand_list[x]:
                                    countand_list[x].append(token)
                x = x+1
            
        countand = 0               
        for c in range(0,3):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(3*countor)
        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "begin":start, "end":end}
            
            
    if n==4:
        countand_list = [[] for i in range(0,4)]
        x = 0
        
        if ex_stop == True:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if (token not in countand_list[x]) and (token not in stops):
                                    countand_list[x].append(token)
                x = x+1
                
        else:
            for s_i in range(0,n):
                for s_j in range(0,n):
                    if s_j != s_i:
                        for token in speaker_vo[s_i]:
                            if token in speaker_vo[s_j]:
                                if token not in countand_list[x]:
                                    countand_list[x].append(token)
                x = x+1
            
        countand = 0               
        for c in range(0,4):
            countand = countand + len(countand_list[c])
        
        countor = len(voor)
        overlap = countand/(4*countor)

        
        vo_group = {"speaker_number": n, "vo_overlap": countand, "vo_total": countor, "total_token_number":token_n, "overlap": overlap,\
                    "s1_and_other": len(countand_list[0]), "s2_and_other": len(countand_list[1]),"s3_and_other": len(countand_list[2]),\
                   "s4_and_other": len(countand_list[3]), 'begin':start, 'end':end}
    
    
    return vo_group
    
    

In [60]:
def number_of_slot_n(speaker_dic,length_u):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    number_slot = int(number_of_u - number_of_u%length_u)/length_u
    return int(number_slot)

In [61]:
def number_of_total_u(speaker_dic):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    return number_of_u

In [62]:
def get_vo_average_speaker_temporal_n(speaker, speaker_dic, length):
    slot_number = number_of_slot_n(speaker_dic, length)
    vo_dic_list = []
    total_u = 0
    
    for s in speaker_dic:
        total_u = total_u + len(s)
    
    for i in range(0,slot_number):
        vo_dic = get_vo_average_speaker_in_window(speaker,speaker_dic, 0, length*(i+1), True)
        vo_dic_list.append(vo_dic)
    
    vo_dic_end = get_vo_average_speaker_in_window(speaker, speaker_dic, 0, total_u, True)
    vo_dic_list.append(vo_dic_end)
    
    return vo_dic_list

In [63]:
out1 = extract_speakers_n(df,3)

In [64]:
out2 = extract_dialogues_dic_n(out1[0],out1[1])

In [65]:
out3 = get_vo(out2[0], out2[1])
print(out3)

{'speaker_number': 2, 'vo_overlap': 802, 'vo_total': 1857, 'total_token_number': 30732, 'overlap': 0.43187937533656434}


In [66]:
out4 = get_vo_pairwise(out2[0], out2[1])
print(out4)

{}


In [67]:
out5 = get_vo_average_speaker(out2[0], out2[1], True)
print(out5)

{'speaker_number': 2, 'vo_overlap': 680, 'vo_total': 1857, 'total_token_number': 30732, 'overlap': 0.366182014001077}


In [68]:
out6 = get_vo_average_speaker_temporal_n(out2[0], out2[1], 50)
for o in out6:
    print(o['overlap'])

0.09032258064516129
0.16049382716049382
0.19536423841059603
0.22074468085106383
0.22511848341232227
0.22964509394572025
0.2438563327032136
0.2461799660441426
0.25
0.25535168195718655
0.2622478386167147
0.26379542395693134
0.27522935779816515
0.28247162673392184
0.28431372549019607
0.2857142857142857
0.2854251012145749
0.28669275929549903
0.28776290630975143
0.29011194029850745
0.2922661870503597
0.30123674911660775
0.3015463917525773
0.3007582139848357
0.30373443983402487
0.3057583130575831
0.3142402545743834
0.3191823899371069
0.32529182879377433
0.3230421686746988
0.32788104089219333
0.32724616508400295
0.32763532763532766
0.3298097251585624
0.33612273361227335
0.33816425120772947
0.3378932968536252
0.33668678739101277
0.33727810650887574
0.3404669260700389
0.34144778987828317
0.34390862944162437
0.34363177805800754
0.3443337484433375
0.344636251541307
0.3464132434089516
0.35112189205579136
0.352622061482821
0.3520958083832335
0.35008870490833827
0.3505275498241501
0.3507549361207898

In [73]:
import pandas as pd
dl = []
temporal_vo = []
import glob,os
path0 = 'tsvfiles/'
path = 'vo_calculate/vo_group2_temporal_average_speaker_ex_sw'
files = glob.glob(os.path.join(path0, "S***.tsv"))
print(len(files))
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,2)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    df_vo_temporal = get_vo_average_speaker_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1],50)
    for d in df_vo_temporal:
        d['file'] = f[len(f)-8:len(f)]
    temporal_vo = pd.DataFrame(df_vo_temporal)
    temporal_vo.to_csv(os.path.join(path, f[len(f)-8:len(f)-4]+'.csv'), index = False)

622


In [70]:
import pandas as pd
dl = []
import glob,os
path0 = 'tsvfiles/'
path = 'vo_calculate/'
files = glob.glob(os.path.join(path0, "S***.tsv"))
print(files)
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,2)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    df_vo = get_vo_average_speaker(dialogue_dic_f[0],dialogue_dic_f[1], True)
    df_vo['file'] = f[len(f)-8:len(f)]
    dl.append(df_vo)

dl_s = pd.DataFrame(dl)
dl_s.to_csv(os.path.join(path,'vo_group_average_speaker_2_ex_sw.csv'), index = False)


['tsvfiles/SGN8.tsv', 'tsvfiles/S72E.tsv', 'tsvfiles/SUPV.tsv', 'tsvfiles/SYEZ.tsv', 'tsvfiles/SRW5.tsv', 'tsvfiles/SG4J.tsv', 'tsvfiles/S2AJ.tsv', 'tsvfiles/S24A.tsv', 'tsvfiles/SL3R.tsv', 'tsvfiles/S5HT.tsv', 'tsvfiles/SZAP.tsv', 'tsvfiles/S4C2.tsv', 'tsvfiles/SXVL.tsv', 'tsvfiles/SJJK.tsv', 'tsvfiles/SXWH.tsv', 'tsvfiles/SDGT.tsv', 'tsvfiles/SZYV.tsv', 'tsvfiles/STUM.tsv', 'tsvfiles/SVCT.tsv', 'tsvfiles/SVBG.tsv', 'tsvfiles/SHJH.tsv', 'tsvfiles/SV4W.tsv', 'tsvfiles/STV6.tsv', 'tsvfiles/SZME.tsv', 'tsvfiles/S32W.tsv', 'tsvfiles/SJE3.tsv', 'tsvfiles/SPXX.tsv', 'tsvfiles/S7QU.tsv', 'tsvfiles/SEM7.tsv', 'tsvfiles/S5ER.tsv', 'tsvfiles/SP5U.tsv', 'tsvfiles/S7FK.tsv', 'tsvfiles/SYDJ.tsv', 'tsvfiles/SUPB.tsv', 'tsvfiles/SRVD.tsv', 'tsvfiles/SN26.tsv', 'tsvfiles/S72Q.tsv', 'tsvfiles/SBS5.tsv', 'tsvfiles/S7P2.tsv', 'tsvfiles/SFPK.tsv', 'tsvfiles/SAZ8.tsv', 'tsvfiles/SAXQ.tsv', 'tsvfiles/SZXQ.tsv', 'tsvfiles/SXAQ.tsv', 'tsvfiles/S87R.tsv', 'tsvfiles/SHJJ.tsv', 'tsvfiles/SVCW.tsv', 'tsvfiles/S6